In [11]:
import requests
import pandas as pd
from datetime import date
from bs4 import BeautifulSoup

def to_int(x):
    if type(x) is int:
        return x
    return int(x.replace(",","").replace(".",""))

In [12]:
url = "https://www.careerlink.vn/vieclam/list?keyword_use=A"
page = requests.get(url)

soup = BeautifulSoup(page.text, 'html.parser')

total_jobs_r = soup.find("div", class_="col-md-7").get_text()

total_jobs = total_jobs_r.split()[2]
total_jobs = to_int(total_jobs)


In [13]:
url = "https://www.careerlink.vn/tim-viec-lam-nhanh"

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html.parser')
soup.prettify()

full_job_inds_dict = {}
full_ind = []

job_inds = soup.find('div', class_='masonry-container').find_all('li')

for ind in job_inds:
    job_ind = ind.find('a').get_text()
    full_ind.append(job_ind)
    job_ind_no = ind.find('span').get_text()
    job_ind_no = to_int(job_ind_no[1:-1])
    full_job_inds_dict[f'{job_ind}'] = job_ind_no

careerlink_result_ind = pd.DataFrame(full_job_inds_dict.items(), columns = ['Industry','No. of Jobs'])
careerlink_result_ind = careerlink_result_ind.sort_values(by=['No. of Jobs'], ascending=False)
careerlink_result_ind

,Industry,No. of Jobs
14,Bán hàng,4540
8,Dịch vụ khách hàng,2662
19,Tư vấn,2655
11,Kế toán / Kiểm toán,2033
4,Thư ký / Hành chánh,1920
33,Kỹ thuật ứng dụng / Cơ khí,1716
18,Tiếp thị,1573
13,Tài chính / Đầu tư,1350
32,Điện / Điện tử,1341
25,CNTT - Phần mềm,1292


In [14]:
#cut last 5 rows of the dataframe and transpose them
#create new dataframe
#merge

url_list = ['https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=I',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=N',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=L',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=T',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=P',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=M',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=D',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=S',
            'https://www.careerlink.vn/vieclam/tim-kiem-viec-lam?keyword_use=A&career_levels=E'
           ]

senority_list = ['Intern', 'Mới đi làm', 'Nhân viên', 'Kỹ thuật viên', 'Trưởng nhóm', 'Quản lý', 'Giám đốc', 'Quản lí cấp cao', 'Điều hành cấp cao']
senority_no = []
senority_dict = {}

for url in url_list:
    
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')

    senority_no_r = soup.find("div", class_="col-md-7").get_text()
    senority_no.append(to_int(senority_no_r.split()[2]))
    
senority_dict = dict(zip(senority_list, senority_no))


In [15]:
Careerlink = {'Total Jobs': total_jobs}

Careerlink.update(senority_dict)

Careerlink_sen = pd.DataFrame(Careerlink, columns=['Total Jobs', 'Intern', 'Mới đi làm', 'Nhân viên', 'Kỹ thuật viên', 
                                    'Trưởng nhóm', 'Quản lý', 'Giám đốc', 'Quản lí cấp cao', 'Điều hành cấp cao'],index=[0])


In [16]:
sum_column_1 = Careerlink_sen["Mới đi làm"] + Careerlink_sen["Nhân viên"] + Careerlink_sen["Kỹ thuật viên"]
sum_column_2 = Careerlink_sen["Trưởng nhóm"] + Careerlink_sen["Quản lý"]
sum_column_3 = Careerlink_sen["Giám đốc"] + Careerlink_sen["Quản lí cấp cao"] + Careerlink_sen["Điều hành cấp cao"]

Careerlink_sen["Junior"] = sum_column_1
Careerlink_sen["Senior"] = sum_column_2
Careerlink_sen['Manager/Director'] = sum_column_3


In [17]:
Careerlink_sen = Careerlink_sen.drop(['Mới đi làm','Nhân viên','Kỹ thuật viên','Trưởng nhóm','Quản lý',
                                      'Giám đốc','Quản lí cấp cao', 'Điều hành cấp cao' ],axis=1)

Careerlink_result_sen = Careerlink_sen.reindex(columns=['Total Jobs','Intern','Junior','Senior',
                                                       'Manager/Director'])
Careerlink_result_sen

,Total Jobs,Intern,Junior,Senior,Manager/Director
0,20118,169,16687,3060,202


In [10]:
from datetime import date
careerlink_result_ind.to_csv(f'careerlink-ind-{date.today()}')
Careerlink_result_sen.to_csv(f'careerlink-sen-{date.today()}')

NameError: name 'Vietnamwork_result_ind' is not defined